In [1]:
import json
import os.path as osp
import numpy as np
import torch
import torch.nn.functional as F

from models.resnet import make_resnet101_base, make_resnet50_base
from utils import set_gpu, pick_vectors

In [2]:
test_set = '2-hops'
test_sets = json.load(open('materials/imagenet-testsets.json', 'r'))
train_wnids = test_sets['train']
test_wnids = test_sets[test_set]

keep_ratio = 0.1
consider_trains = True
print('test set: {}, {} classes, ratio={}'
      .format(test_set, len(test_wnids), keep_ratio))
print('consider train classifiers: {}'.format(consider_trains))

# pred = 'save/gcn-att/best_val.pred'
# pred = 'save/gcn-dense-aux/best_val.pred'
# pred_file = torch.load(pred)
# pred_wnids = pred_file['wnids']
# pred_vectors = pred_file['pred']
# pred_dic = dict(zip(pred_wnids, pred_vectors))
# pred_vectors = pick_vectors(pred_dic, train_wnids + test_wnids, is_tensor=True).cuda()  # 得到train和test的预测的分类器

# pred_vectors = pred_vectors.cuda()

n = len(train_wnids)
m = len(test_wnids)

# cnn = eval(f'make_resnet101_base()')
# cnn.load_state_dict(torch.load(f'materials/resnet101-base.pth'))
# cnn = cnn.cuda()
# cnn.eval()

TEST_TRAIN = True

s_hits = torch.FloatTensor([0, 0, 0, 0, 0]).cuda() # top 1 2 5 10 20
s_tot = 0

results = {}


test set: 2-hops, 1549 classes, ratio=0.1
consider train classifiers: True


In [3]:
# 通过预提取保存到本地的image feature进行test
npy_path = '/home/zzc/datasets/imagenet_feats'
def test_on_subset_local(wnid, n, pred_vectors, all_label,
                   consider_trains):
    top = [1, 2, 5, 10, 20]
    hits = torch.zeros(len(top)).cuda()
    tot = 0
    
    feat = torch.FloatTensor(np.load(os.path.join(npy_path, f'{wnid}.npy'))).cuda()
    feat = torch.cat([feat, torch.ones(len(feat)).view(-1, 1).cuda()], dim=1)
    fcs = pred_vectors.t()

    table = torch.matmul(feat, fcs)
    if not consider_trains:
        table[:, :n] = -1e18  # 将train class部分置0

    gth_score = table[:, all_label].repeat(table.shape[1], 1).t() # 该batch内，target label的score
    rks = (table >= gth_score).sum(dim=1)  # 若其他类预测分数大于target类，则说明分类错误；若rk==1，则说明分类正确。

    assert (table[:, all_label] == gth_score[:, all_label]).min() == 1

    for i, k in enumerate(top):
        hits[i] += (rks <= k).sum().item()
    tot += len(feat)

    return hits, tot

In [17]:
import os

test_set = '2-hops'
test_wnids = test_sets[test_set]

save = 'gcn-att-r-4layers' # gcn-att
pred = f'{save}/best_val.pred'
pred_file = torch.load(pred)
pred_wnids = pred_file['wnids']
pred_vectors = pred_file['pred']
pred_dic = dict(zip(pred_wnids, pred_vectors))
pred_vectors = pick_vectors(pred_dic, train_wnids + test_wnids, is_tensor=True).cuda()  # 得到train和test的预测的分类器

s_hits = torch.FloatTensor([0, 0, 0, 0, 0]).cuda() # top 1 2 5 10 20
s_tot = 0

# for idx, wnid in enumerate(train_wnids, 1):
#     if not os.path.exists(os.path.join(npy_path, f'{wnid}.npy')):
#         continue    
#     hits, tot = test_on_subset_local(wnid, n, pred_vectors, idx - 1,
#                                consider_trains=True)

#     s_hits += hits
#     s_tot += tot

# #     print('{}/{}, {}:'.format(idx, len(train_wnids), wnid), end=' ')
# #     for i in range(len(hits)):
# #         print('{:.0f}%({:.2f}%)'
# #               .format(hits[i] / tot * 100, s_hits[i] / s_tot * 100), end=' ')
#     print('x{}({})'.format(tot, s_tot))
    
for idx, wnid in enumerate(test_wnids, 1):
    if not os.path.exists(os.path.join(npy_path, f'{wnid}.npy')):
        continue    
    hits, tot = test_on_subset_local(wnid, n, pred_vectors, n+idx-1,
                               consider_trains=False)

    s_hits += hits
    s_tot += tot

#     print('{}/{}, {}:'.format(idx, len(test_wnids), wnid), end=' ')
#     for i in range(len(hits)):
#         print('{:.0f}%({:.2f}%)'
#               .format(hits[i] / tot * 100, s_hits[i] / s_tot * 100), end=' ')
#     print('x{}({})'.format(tot, s_tot))

print('summary:', end=' ')
for s_hit in s_hits:
    print('{:.2f}%'.format(s_hit / s_tot * 100), end=' ')
# print('total {}'.format(s_tot))

summary: 21.50% 34.60% 52.80% 64.35% 73.28% 

In [ ]:
nell-edges-all-word1300:
train: summary: 56.45% 76.70% 90.16% 94.69% 97.19% total 129022
2-hops-zsl: 25.57% 39.26% 58.74% 70.91% 80.18% total 130590
2-hops-gzsl:11.33% 25.92% 48.93% 63.25% 74.54% total 130590
    
gcn-att:
2-hops:15.62% 25.53% 40.92% 53.42% 64.92% total 130590
2-hops-gzsl:6.97% 14.76% 30.32% 43.12% 56.28% total 130590

5, 20000:
    18.87% 30.09% 45.99% 56.34% 65.16% 
    8.96% 19.07% 35.35% 47.07% 57.39% 
5,10000
20.53% 32.21% 49.08% 60.86% 70.16%
4,20000
20.13% 31.92% 48.75% 60.18% 69.62%

gcn-att-r: 
    padding 5, neighs 10000
    2-hops: 18.60% 29.10% 43.93% 53.81% 61.55% total 130590
    1.85% 12.09% 29.43% 41.14% 50.68% total 130590
    padding 15, neighs 5000
    14.86% 22.80% 36.30% 46.93% 57.20% total 130590
    
    10, 5000, 使用aux_attention 
    22.00% 33.74% 50.53% 61.93% 70.83%
    9.49% 22.08% 40.90% 53.63% 64.32% 

In [ ]:
torch.argsort(table, descending=True, dim=1)

In [ ]:
def f(x):
    m = np.mean(x, axis=0)
    res = np.sum([np.sum([abs(a-b) for a, b in zip(xx, m)])/len(xx) for xx in x])/len(x)
    return res

In [ ]:
import scipy.io as sio
from sklearn.preprocessing import scale 
path = '/home/zzc/exp/Hierarchically_Learning_The_Discriminative_Features_For_Zero_Shot_Learning/data'

mat = sio.loadmat(os.path.join(path, f'AWA2_data','att_splits.mat'))
att = mat['original_att'].T
att

In [ ]:
ds = ['AWA2', 'CUB', 'APY', 'SUN']
path = '/home/zzc/exp/Hierarchically_Learning_The_Discriminative_Features_For_Zero_Shot_Learning/data'
for d in ds:
    mat = sio.loadmat(os.path.join(path, f'{d}_data','att_splits.mat'))
    att = mat['original_att'].T
#     std = np.std(scale(att, axis=1), axis=0).mean()
    print(d, f(att))

In [ ]:
mat['original_att']

In [ ]:
def infor(data):
    a = pd.value_counts(data) / len(data)
    return sum(np.log2(a) * a * (-1))